In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import shapely
from shapely.ops import transform

In [135]:
# this is the modified geojson with reduced coordinate precision
input_file = r"C:\Users\haria\OneDrive\Desktop\Omdena-Project-Canopy1\output\image_Shifting_cultivation_6.geojson"
output_file = r"C:\Users\haria\OneDrive\Desktop\Omdena-Project-Canopy1\output\image_Shifting_cultivation_6_dissolve.geojson"
output_file2 = r"C:\Users\haria\OneDrive\Desktop\Omdena-Project-Canopy1\output\image_Shifting_cultivation_6_simple.geojson"

In [136]:
gdf = gpd.read_file(input_file)

In [137]:
gdf.head()

,raster_val,geometry
0,470.0,"POLYGON ((28.71061 0.28575, 28.71061 0.28566, ..."
1,479.0,"POLYGON ((28.71070 0.28575, 28.71070 0.28566, ..."
2,497.0,"POLYGON ((28.71079 0.28575, 28.71079 0.28566, ..."
3,563.0,"POLYGON ((28.71088 0.28575, 28.71088 0.28566, ..."
4,638.0,"POLYGON ((28.71096 0.28575, 28.71096 0.28566, ..."


In [138]:
raster_val = []
raster_val = pd.DataFrame(pd.unique(gdf["raster_val"].sort_values()), dtype = float, columns=['a'])
raster_val.reset_index(inplace=True)
raster_val.rename(columns={"index": "idx"},inplace=True)
raster_val.head()

,idx,a
0,0,194.0
1,1,196.0
2,2,200.0
3,3,201.0
4,4,203.0


In [139]:
gdf["new_val"] = 0

gdf = pd.merge(gdf, raster_val, left_on = 'raster_val', right_on = 'a', how='outer')

gdf.head()

,raster_val,geometry,new_val,idx,a
0,470.0,"POLYGON ((28.71061 0.28575, 28.71061 0.28566, ...",0,270,470.0
1,470.0,"POLYGON ((28.73333 0.28566, 28.73333 0.28557, ...",0,270,470.0
2,470.0,"POLYGON ((28.72040 0.28557, 28.72040 0.28548, ...",0,270,470.0
3,470.0,"POLYGON ((28.73136 0.28530, 28.73136 0.28522, ...",0,270,470.0
4,470.0,"POLYGON ((28.73172 0.28486, 28.73172 0.28477, ...",0,270,470.0


In [140]:
gdf.new_val = (gdf["idx"]/max(gdf["idx"])*100).astype(int)
gdf.drop(['idx','a'], axis=1, inplace=True)
gdf.head()

,raster_val,geometry,new_val
0,470.0,"POLYGON ((28.71061 0.28575, 28.71061 0.28566, ...",34
1,470.0,"POLYGON ((28.73333 0.28566, 28.73333 0.28557, ...",34
2,470.0,"POLYGON ((28.72040 0.28557, 28.72040 0.28548, ...",34
3,470.0,"POLYGON ((28.73136 0.28530, 28.73136 0.28522, ...",34
4,470.0,"POLYGON ((28.73172 0.28486, 28.73172 0.28477, ...",34


In [141]:
import mapclassify


In [142]:
# Create a Natural Breaks classifier
mapclassify.NaturalBreaks(y=gdf.new_val, k=5)

NaturalBreaks

    Interval       Count
------------------------
[  0.00,  11.00] | 77635
( 11.00,  21.00] | 49983
( 21.00,  36.00] | 15944
( 36.00,  55.00] |  9560
( 55.00, 100.00] |  5272

In [143]:
# Classify the data
# gdf.Class = gdf.new_val.apply(mapclassify.NaturalBreaks(y=gdf.new_val, k=5))
gdf['Class'] = gdf.new_val.apply(mapclassify.NaturalBreaks(y=gdf.new_val, k=5))

# Let's see what we have
gdf.head()

,raster_val,geometry,new_val,Class
0,470.0,"POLYGON ((28.71061 0.28575, 28.71061 0.28566, ...",34,[2]
1,470.0,"POLYGON ((28.73333 0.28566, 28.73333 0.28557, ...",34,[2]
2,470.0,"POLYGON ((28.72040 0.28557, 28.72040 0.28548, ...",34,[2]
3,470.0,"POLYGON ((28.73136 0.28530, 28.73136 0.28522, ...",34,[2]
4,470.0,"POLYGON ((28.73172 0.28486, 28.73172 0.28477, ...",34,[2]


In [144]:
# input_array

In [145]:
def flatten(input_array):
    result_array = []
    for element in input_array:
        #print(element[0])
        # result_array.append(element[0]+1)
        result_array.append(element+1)
    return result_array

In [146]:
results = flatten(gdf.Class)
results = pd.DataFrame(flatten(gdf.Class))
gdf.Class = results
gdf.head()

,raster_val,geometry,new_val,Class
0,470.0,"POLYGON ((28.71061 0.28575, 28.71061 0.28566, ...",34,3
1,470.0,"POLYGON ((28.73333 0.28566, 28.73333 0.28557, ...",34,3
2,470.0,"POLYGON ((28.72040 0.28557, 28.72040 0.28548, ...",34,3
3,470.0,"POLYGON ((28.73136 0.28530, 28.73136 0.28522, ...",34,3
4,470.0,"POLYGON ((28.73172 0.28486, 28.73172 0.28477, ...",34,3


In [147]:
gdf.Class

0         3
1         3
2         3
3         3
4         3
         ..
158389    5
158390    5
158391    5
158392    5
158393    5
Name: Class, Length: 158394, dtype: int64

In [148]:
# gdf_class = gdf.dissolve(by='Class')
# gdf_class = gdf.dissolve(results)
gdf_class = gdf.dissolve('Class')


In [149]:
gdf_class.drop(['raster_val', 'new_val'], axis=1, inplace=True)
gdf_class.reset_index(inplace=True)
gdf_class


,Class,geometry
0,1,"MULTIPOLYGON (((28.71105 0.25324, 28.71096 0.2..."
1,2,"MULTIPOLYGON (((28.71150 0.25252, 28.71150 0.2..."
2,3,"MULTIPOLYGON (((28.71159 0.25135, 28.71150 0.2..."
3,4,"MULTIPOLYGON (((28.71114 0.25144, 28.71114 0.2..."
4,5,"MULTIPOLYGON (((28.71079 0.25180, 28.71088 0.2..."


In [150]:
gdf_class2 = gdf_class.explode(index_parts=True)

In [151]:
gdf_class2

Class                                           geometry
0 0        1  POLYGON ((28.71105 0.25324, 28.71096 0.25324, ...
  1        1  POLYGON ((28.71079 0.25359, 28.71079 0.25350, ...
  2        1  POLYGON ((28.71061 0.25665, 28.71070 0.25665, ...
  3        1  POLYGON ((28.71088 0.26662, 28.71096 0.26662, ...
  4        1  POLYGON ((28.71070 0.26752, 28.71070 0.26761, ...
...      ...                                                ...
4 496      5  POLYGON ((28.74896 0.28045, 28.74905 0.28045, ...
  497      5  POLYGON ((28.74896 0.28045, 28.74887 0.28045, ...
  498      5  POLYGON ((28.74932 0.28108, 28.74941 0.28108, ...
  499      5  POLYGON ((28.74851 0.28099, 28.74842 0.28099, ...
  500      5  POLYGON ((28.74941 0.28126, 28.74950 0.28126, ...

[10118 rows x 2 columns]

In [152]:
len(gdf)

158394

In [153]:
#save the file:
gdf_class2.to_file(output_file)

In [154]:
from shapely.geometry import Point, LineString
#EPSG:3857
#gdf_class2.to_file(output_file2)

gdf_class2 = gdf_class2.to_crs(3857)

In [155]:
#simplifying to a meter:
gdf_class2.geometry=gdf_class2.simplify(1)
gdf_class2 = gdf_class2.to_crs(4326)

In [156]:
gdf_class2.to_file(output_file2)

In [157]:
gdf_class2.tail()

Class                                           geometry
4 496      5  POLYGON ((28.74896 0.28045, 28.74932 0.28045, ...
  497      5  POLYGON ((28.74896 0.28045, 28.74887 0.28045, ...
  498      5  POLYGON ((28.74932 0.28108, 28.74941 0.28108, ...
  499      5  POLYGON ((28.74851 0.28099, 28.74833 0.28099, ...
  500      5  POLYGON ((28.74941 0.28126, 28.74950 0.28126, ...